Discover Mechanics
==========

Let's do some Monte Carlos for Discover as well.  Some ones to try:

* Tomb Spider (Discover a Beast)
* Jewled Scarab (Discover a 3 Cost Card)
* Dark Peddler (Discover a 1-Cost Card - Warlock Only)
* Ethereal Conjurer (Discover a Spell - Mage Only)

In [20]:
# Load up all the cards
from helpers.data import load_dataframe, player_classes
from helpers.discover import discover
from collections import Counter
import numpy as np

df = load_dataframe()

In [62]:
# Tomb Spider (Discover a Beast)
# First, let's filter down to just the beasts
beasts = df[df.race == "Beast"]

# let's get the initial optimized dataframes for each class
beasts_by_class = dict()
for cls in player_classes:
    discarded_best, discarded_picks, optimized_df = discover(cls, beasts)
    beasts_by_class[cls] = optimized_df

def _discover_monte_carlo(cards):
    by_class = dict()
    for cls in player_classes:
        discarded_best, discarded_picks, optimized_df = discover(cls, cards)
        by_class[cls] = optimized_df
        
    def perform_monte_carlo(cls, n=10000):
        best_counter = Counter()
        picks_counter = Counter()
        for i in range(n):
            best, picks, discard = discover(cls, by_class[cls], needs_weights=False)
            best_counter.update([best["name"]])
            picks_counter.update(picks["name"].values)
        return best_counter, picks_counter
    
    func = perform_monte_carlo
    func.by_class = by_class
    return func

# our function for monte-carloing the tomb spider
tomb_spider_monte_carlo = _discover_monte_carlo(beasts)

In [66]:
best, picks = tomb_spider_monte_carlo("Hunter", n=10000)
best.most_common(30)

[(u'Savannah Highmane', 1394),
 (u'King Krush', 1255),
 (u"Gahz'rilla", 1077),
 (u"King's Elekk", 910),
 (u'Dreadscale', 814),
 (u'Webspinner', 657),
 (u'Stampeding Kodo', 314),
 (u'Core Rager', 313),
 (u'Scavenging Hyena', 275),
 (u'Stranglethorn Tiger', 275),
 (u'Haunted Creeper', 274),
 (u'Maexxna', 212),
 (u'King of Beasts', 197),
 (u'Emperor Cobra', 172),
 (u'Tomb Spider', 167),
 (u'King Mukla', 149),
 (u'Dire Wolf Alpha', 146),
 (u'The Beast', 130),
 (u'Captured Jormungar', 130),
 (u'Lost Tallstrider', 129),
 (u'Jeweled Scarab', 128),
 (u'Tundra Rhino', 121),
 (u'Jungle Panther', 117),
 (u'Armored Warhorse', 106),
 (u'River Crocolisk', 99),
 (u'Bloodfen Raptor', 93),
 (u'Timber Wolf', 59),
 (u'Ironbeak Owl', 57),
 (u'Ironfur Grizzly', 53),
 (u'Oasis Snapjaw', 35)]

In [67]:
# The Average Best Score is Actually much better than the Average Score
scores = []
for card, count in best.items():
    score = beasts[beasts["name"] == card].iloc[0].Hunter_Tier
    scores.extend([score] * count)

avg_best_score = np.mean(scores)
avg_best_score, tomb_spider_monte_carlo.by_class["Hunter"].Hunter_Tier.mean()

(74.159199999999998, 54.358974358974358)

In [55]:
# Jeweled Scarab
threes = df[df.cost == 3]

# our monte carlo function
jeweled_scarab_monte_carlo = _discover_monte_carlo(threes)

In [61]:
best, picks = jeweled_scarab_monte_carlo("Mage", n=10000)
best.most_common(30)

[(u'Spellslinger', 755),
 (u'Arcane Intellect', 691),
 (u'Flamewaker', 602),
 (u'Kirin Tor Mage', 512),
 (u'Forgotten Torch', 416),
 (u'Mirror Entity', 409),
 (u'Polymorph: Boar', 283),
 (u'Soot Spewer', 256),
 (u'Harvest Golem', 248),
 (u'Argent Horserider', 235),
 (u'Fjola Lightbane', 231),
 (u'Scarlet Crusader', 229),
 (u'Shattered Sun Cleric', 222),
 (u'Spider Tank', 219),
 (u'Eydis Darkbane', 209),
 (u'Blood Knight', 202),
 (u'Ogre Brute', 200),
 (u'Earthen Ring Farseer', 197),
 (u'Shade of Naxxramas', 189),
 (u'Effigy', 188),
 (u'Big Game Hunter', 187),
 (u'Mind Control Tech', 184),
 (u'Counterspell', 182),
 (u'Acolyte of Pain', 176),
 (u'Imp Master', 169),
 (u'Saboteur', 167),
 (u'Emperor Cobra', 145),
 (u'King Mukla', 145),
 (u'Silver Hand Regent', 134),
 (u'Silent Knight', 119)]

In [34]:
# Ethereal Conjurer
spells = df[df.type == "Spell"]
discarded_best, discarded_picks, spells = discover("Mage", spells)

def ethereal_monte_carlo(n=10000):
    best_counter = Counter()
    picks_counter = Counter()
    for i in range(n):
        best, picks, discard = discover("Mage", spells, needs_weights=False)
        best_counter.update([best["name"]])
        picks_counter.update(picks["name"].values)
    return best_counter, picks_counter

In [38]:
best, picks =ethereal_monte_carlo(n=10000)

[(u'Flamestrike', 1041),
 (u'Fireball', 1010),
 (u'Blizzard', 912),
 (u'Flamecannon', 769),
 (u'Frostbolt', 769),
 (u'Polymorph', 723),
 (u'Pyroblast', 615),
 (u'Flame Lance', 550),
 (u'Unstable Portal', 505),
 (u'Arcane Intellect', 486),
 (u'Arcane Blast', 410),
 (u'Forgotten Torch', 340),
 (u'Mirror Entity', 321),
 (u"Dragon's Breath", 283),
 (u'Polymorph: Boar', 252),
 (u'Effigy', 232),
 (u'Arcane Missiles', 202),
 (u'Arcane Explosion', 130),
 (u'Counterspell', 123),
 (u'Cone of Cold', 98),
 (u'Duplicate', 86),
 (u'Mirror Image', 49),
 (u'Echo of Medivh', 39),
 (u'Vaporize', 25),
 (u'Spellbender', 20),
 (u'Frost Nova', 6),
 (u'Ice Barrier', 4)]

In [46]:
best.most_common(5), "  ", picks.most_common(30)

([(u'Flamestrike', 1041),
  (u'Fireball', 1010),
  (u'Blizzard', 912),
  (u'Flamecannon', 769),
  (u'Frostbolt', 769)],
 '  ',
 [(u'Frost Nova', 1109),
  (u'Arcane Intellect', 1097),
  (u'Fireball', 1086),
  (u'Flame Lance', 1064),
  (u'Polymorph', 1062),
  (u'Counterspell', 1062),
  (u'Blizzard', 1058),
  (u'Ice Block', 1054),
  (u'Arcane Explosion', 1049),
  (u'Spellbender', 1045),
  (u'Arcane Blast', 1042),
  (u'Arcane Missiles', 1041),
  (u'Flamestrike', 1041),
  (u'Mirror Image', 1035),
  (u'Effigy', 1030),
  (u'Frostbolt', 1027),
  (u"Dragon's Breath", 1026),
  (u'Duplicate', 1024),
  (u'Echo of Medivh', 1019),
  (u'Ice Lance', 1017),
  (u'Forgotten Torch', 1016),
  (u'Pyroblast', 1013),
  (u'Cone of Cold', 1013),
  (u'Polymorph: Boar', 1012),
  (u'Unstable Portal', 1007),
  (u'Flamecannon', 1006),
  (u'Vaporize', 1002),
  (u'Ice Barrier', 992),
  (u'Mirror Entity', 951)])

In [68]:
# Dark Peddler
ones = df[df.cost == 1]
discarded_best, discarded_picks, ones = discover("Warlock", ones)

def dark_peddler_monte_carlo(n=10000):
    best_counter = Counter()
    picks_counter = Counter()
    for i in range(n):
        best, picks, discard = discover("Warlock", ones, needs_weights=False)
        best_counter.update([best["name"]])
        picks_counter.update(picks["name"].values)
    return best_counter, picks_counter

In [71]:
best, picks = dark_peddler_monte_carlo(n=10000)
best.most_common(30)

[(u'Mortal Coil', 1856),
 (u'Voidwalker', 1450),
 (u'Power Overwhelming', 1358),
 (u'Flame Imp', 1050),
 (u'Soulfire', 1012),
 (u'Blood Imp', 635),
 (u'Zombie Chow', 495),
 (u'Abusive Sergeant', 264),
 (u'Worgen Infiltrator', 171),
 (u'Argent Squire', 170),
 (u'Elven Archer', 149),
 (u'Corruption', 143),
 (u'Young Priestess', 130),
 (u'Clockwork Gnome', 128),
 (u'Gadgetzan Jouster', 116),
 (u'Voodoo Doctor', 100),
 (u'Southsea Deckhand', 89),
 (u'Stonetusk Boar', 84),
 (u'Leper Gnome', 82),
 (u'Tournament Attendee', 73),
 (u'Lowly Squire', 71),
 (u'Murloc Raider', 68),
 (u'Goldshire Footman', 57),
 (u'Injured Kvaldir', 55),
 (u'Cogmaster', 48),
 (u'Bloodsail Corsair', 42),
 (u'Shieldbearer', 26),
 (u'Hungry Crab', 20),
 (u'Lightwarden', 15),
 (u'Undertaker', 14)]

In [72]:
picks.most_common(30)

[(u'Flame Imp', 1987),
 (u'Reliquary Seeker', 1948),
 (u'Mortal Coil', 1930),
 (u'Soulfire', 1917),
 (u'Blood Imp', 1879),
 (u'Voidwalker', 1878),
 (u'Corruption', 1846),
 (u'Power Overwhelming', 1801),
 (u'Sir Finley Mrrgglton', 566),
 (u'Abusive Sergeant', 546),
 (u'Southsea Deckhand', 534),
 (u'Lowly Squire', 523),
 (u'Shieldbearer', 522),
 (u'Injured Kvaldir', 519),
 (u'Young Dragonhawk', 514),
 (u'Leper Gnome', 506),
 (u'Murloc Tidecaller', 498),
 (u'Elven Archer', 497),
 (u'Zombie Chow', 495),
 (u'Dragon Egg', 493),
 (u'Murloc Raider', 490),
 (u'Lightwarden', 490),
 (u'Hungry Crab', 486),
 (u'Grimscale Oracle', 486),
 (u'Tournament Attendee', 486),
 (u'Undertaker', 485),
 (u'Stonetusk Boar', 485),
 (u'Secretkeeper', 484),
 (u'Goldshire Footman', 482),
 (u'Clockwork Gnome', 482)]